### Even within the same store, there may have different regular prices for the same SKU. 

In [1]:
import pandas as pd
data_type = pd.read_csv('data_type.csv')
data_type.fillna('',inplace=True)

In [2]:
data_type.head()

,StoreID,TransactionID,Line,Sales,Count,Date,SKU,Type
0,A29,11-7270-00007-00057,36,850.0,1.0,3/11/12 00:00,A21773,
1,A29,11-7270-00007-00057,39,1500.0,1.0,3/11/12 00:00,A18009,
2,A29,11-7270-00007-00057,40,1500.0,1.0,3/11/12 00:00,A25592,
3,A29,11-7270-00007-00057,41,1600.0,1.0,3/11/12 00:00,A1449,
4,A29,11-7270-00007-00057,42,1600.0,1.0,3/11/12 00:00,A1449,


In [3]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [39]:
q = '''
    select
        a.*
    from data_type a
    join
    (select distinct StoreID, SKU from data_type where Count=0) b
    on a.StoreID=b.StoreID and a.SKU=b.SKU
    '''
data30 = pysqldf(q)

In [40]:
#Select SKUs within a specific store where number of distinct regular prices is bigger than 1
q = '''
    select StoreID, SKU, count(price)
    from
    (select distinct StoreID, SKU, Sales/Count as price
    from data30
    where Count>0)
    group by StoreID,SKU
    having count(price)>1
    '''
data31 = pysqldf(q)

In [41]:
data31.head()

,StoreID,SKU,count(price)
0,A01,A6873,2
1,A01,A7257,2
2,A01,A8076,2
3,A02,A13228,2
4,A02,A2605,2


In [43]:
data31.shape

(412, 3)

#### Different Scenarios

In [52]:
data_type[(data_type.SKU=='A6873')&(data_type.StoreID=='A01')]

,StoreID,TransactionID,Line,Sales,Count,Date,SKU,Type
54851,A01,12-7149-00001-00035,3,3018.0,1.0,3/12/12 00:00,A6873,
58534,A01,12-7149-00009-00037,1,3018.0,1.0,3/12/12 00:00,A6873,
60535,A01,12-7149-00011-00113,3,1995.0,1.0,3/12/12 00:00,A6873,
64033,A01,12-7149-00006-00313,4,3018.0,1.0,3/12/12 00:00,A6873,
65828,A01,12-7149-00004-00523,10,3018.0,1.0,3/12/12 00:00,A6873,
65829,A01,12-7149-00004-00523,11,-269.0,0.0,3/12/12 00:00,A6873,


In [53]:
data_type[(data_type.SKU=='A7257')&(data_type.StoreID=='A01')]

,StoreID,TransactionID,Line,Sales,Count,Date,SKU,Type
51935,A01,12-7149-00002-00389,3,2095.0,1.0,3/12/12 00:00,A7257,
54420,A01,12-7149-00002-00693,1,2095.0,1.0,3/12/12 00:00,A7257,
54423,A01,12-7149-00002-00693,6,2095.0,1.0,3/12/12 00:00,A7257,
54425,A01,12-7149-00002-00693,11,-2095.0,-1.0,3/12/12 00:00,A7257,Regular_Return
54426,A01,12-7149-00002-00693,12,1250.0,1.0,3/12/12 00:00,A7257,
54427,A01,12-7149-00002-00693,14,-2095.0,-1.0,3/12/12 00:00,A7257,Regular_Return
54428,A01,12-7149-00002-00693,15,1250.0,1.0,3/12/12 00:00,A7257,
59193,A01,12-7149-00005-00358,9,2095.0,1.0,3/12/12 00:00,A7257,
59194,A01,12-7149-00005-00358,10,-845.0,0.0,3/12/12 00:00,A7257,
59195,A01,12-7149-00005-00358,11,2095.0,1.0,3/12/12 00:00,A7257,


#### Practice: For Store A01 and SKU A7257 where Sales=1250, do the imputation to plug in the discounted rows where count=0 and sales is a negative number

In [112]:
#Do the imputation to plug in the discounted rows where count=0 and sales is a negative number
data_type = data_type.set_value(54428,'Count',0)
data_type = data_type.set_value(54428,'Sales',-845)
data_type.iloc[54428]

StoreID                          A01
TransactionID    12-7149-00002-00693
Line                              15
Sales                           -845
Count                              0
Date                   3/12/12 00:00
SKU                            A7257
Type                                
Name: 54428, dtype: object

In [130]:
data = []
# always inserting new rows at the first position - last row will be always on top    
data.insert(0, {'StoreID': 'A01', 'TransactionID': '12-7149-00002-00693', 'Line': 15,'Sales':2095,'Count':1,'Date':'3/12/12 00:00','SKU':'A7257','Type':''})
data_type= pd.concat([data_type[:54428],pd.DataFrame(data), data_type[54428:]], ignore_index=True)

In [131]:
data_type[(data_type.SKU=='A7257')&(data_type.StoreID=='A01')]

,Count,Date,Line,SKU,Sales,StoreID,TransactionID,Type
51935,1.0,3/12/12 00:00,3,A7257,2095.0,A01,12-7149-00002-00389,
54420,1.0,3/12/12 00:00,1,A7257,2095.0,A01,12-7149-00002-00693,
54423,1.0,3/12/12 00:00,6,A7257,2095.0,A01,12-7149-00002-00693,
54425,-1.0,3/12/12 00:00,11,A7257,-2095.0,A01,12-7149-00002-00693,Regular_Return
54426,1.0,3/12/12 00:00,12,A7257,1250.0,A01,12-7149-00002-00693,
54427,-1.0,3/12/12 00:00,14,A7257,-2095.0,A01,12-7149-00002-00693,Regular_Return
54428,1.0,3/12/12 00:00,15,A7257,2095.0,A01,12-7149-00002-00693,
54429,0.0,3/12/12 00:00,15,A7257,-845.0,A01,12-7149-00002-00693,
59194,1.0,3/12/12 00:00,9,A7257,2095.0,A01,12-7149-00005-00358,
59195,0.0,3/12/12 00:00,10,A7257,-845.0,A01,12-7149-00005-00358,


#### Other Scenarios

In [56]:
data_type[(data_type.SKU=='A8076')&(data_type.StoreID=='A01')][:15]

,StoreID,TransactionID,Line,Sales,Count,Date,SKU,Type
51603,A01,12-7149-00002-00312,1,2500.0,1.0,3/12/12 00:00,A8076,
51883,A01,12-7149-00002-00376,1,2500.0,1.0,3/12/12 00:00,A8076,
52848,A01,12-7149-00002-00193,9,2500.0,1.0,3/12/12 00:00,A8076,
52849,A01,12-7149-00002-00193,10,-200.0,0.0,3/12/12 00:00,A8076,
52850,A01,12-7149-00002-00193,11,2500.0,1.0,3/12/12 00:00,A8076,
52851,A01,12-7149-00002-00193,12,-200.0,0.0,3/12/12 00:00,A8076,
52858,A01,12-7149-00002-00193,21,2500.0,1.0,3/12/12 00:00,A8076,
52859,A01,12-7149-00002-00193,22,-200.0,0.0,3/12/12 00:00,A8076,
54705,A01,12-7149-00001-00005,1,3510.0,1.0,3/12/12 00:00,A8076,
54706,A01,12-7149-00001-00005,2,-1210.0,0.0,3/12/12 00:00,A8076,


In [51]:
data_type[(data_type.SKU=='A13228')&(data_type.StoreID=='A02')]

,StoreID,TransactionID,Line,Sales,Count,Date,SKU,Type
73056,A02,12-2908-00003-00187,6,9995.0,1.0,3/12/12 00:00,A13228,
73057,A02,12-2908-00003-00187,7,-2495.0,0.0,3/12/12 00:00,A13228,
77563,A02,12-2908-00013-00055,6,3750.0,1.0,3/12/12 00:00,A13228,


<br><br><br>

In [ ]:
#Find out those rows where regular price equals to price after discount for specfic SKU within the same store

In [72]:
data30.head(3)

,StoreID,TransactionID,Line,Sales,Count,Date,SKU,Type
0,A29,11-7270-00007-00057,57,1695.0,1.0,3/11/12 00:00,A13081,
1,A29,11-7270-00007-00057,58,2095.0,1.0,3/11/12 00:00,A5985,
2,A29,11-7270-00007-00057,62,2095.0,1.0,3/11/12 00:00,A256,


In [68]:
q = '''
    select
        distinct a.StoreID, a.SKU, sum(a.Sales)/sum(a.Count) as Discount_Price
    from data30 a
    join
    (select distinct TransactionID, SKU from data30 where Count=0) b
    on a.TransactionID=b.TransactionID and a.SKU=b.SKU
    group by a.TransactionID, a.SKU
    '''
data32 = pysqldf(q)

In [71]:
data32[(data32.SKU=='A7257')].head()

,StoreID,SKU,Discount_Price
3729,A60,A7257,1250.0
4065,A60,A7257,1221.0
12104,A35,A7257,1380.0
12158,A35,A7257,1221.0
15656,A49,A7257,1250.0


In [80]:
q = '''
    select
        distinct a.StoreID, a.SKU, sum(a.Sales)/sum(a.Count) as Regular_Price
    from (select * from data30 where Count>0) a
    left join
    (select distinct SKU, TransactionID from data30 where Count=0) b
    on a.TransactionID=b.TransactionID and a.SKU=b.SKU
    where b.TransactionID is null
    group by a.TransactionID, a.SKU
    '''
data33 = pysqldf(q)

In [81]:
data33.head()

,StoreID,SKU,Regular_Price
0,A60,A25083,2195.0
1,A60,A14671,2195.0
2,A60,A7880,2195.0
3,A60,A1092,1312.0
4,A60,A15452,1200.0


In [86]:
q='''
   select a.StoreID, a.SKU, a.Discount_Price
   from data32 a
   join data33 b
   on a.StoreID=b.StoreID and a.SKU=b.SKU and a.Discount_Price=b.Regular_Price  
  '''
data34=pysqldf(q)

In [87]:
data34.head()

,StoreID,SKU,Discount_Price
0,A65,A7241,1000.0
1,A65,A10705,1000.0
2,A65,A7367,1000.0
3,A65,A22213,1000.0
4,A65,A15325,1250.0


In [88]:
data34.shape

(94, 3)

In [92]:
data_type[(data_type.SKU=='A10705')&(data_type.StoreID=='A65')][-7:]

,StoreID,TransactionID,Line,Sales,Count,Date,SKU,Type
811799,A65,12-3572-00001-00198,9,1525.0,1.0,3/12/12 00:00,A10705,
811800,A65,12-3572-00001-00198,10,-525.0,0.0,3/12/12 00:00,A10705,
811819,A65,12-3572-00001-00202,3,1525.0,1.0,3/12/12 00:00,A10705,
811820,A65,12-3572-00001-00202,4,-525.0,0.0,3/12/12 00:00,A10705,
816426,A65,12-3572-00002-00488,14,1000.0,1.0,3/12/12 00:00,A10705,
817830,A65,12-3572-00009-00143,1,1000.0,1.0,3/12/12 00:00,A10705,
820236,A65,12-3572-00001-00753,3,1000.0,1.0,3/12/12 00:00,A10705,


In [96]:
data_type.shape

(1041577, 8)